In [ ]:
import csv
import pickle
import json
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from mouselab.mouselab import MouselabEnv
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy
import math
import sklearn.cluster
from math import floor, ceil

# Reading Data from Files

The anonymized participant data can be found on the GitHub repository for this project in the folder 'anonymized_data'. Then, the remainder of the cells in the notebook can be executed.

In [ ]:
fileprefix = "final" # pilot_#, final
datafolder = '../results/anonymized_data/'

all_part_files = [filename for filename in os.listdir(datafolder) if filename.startswith(fileprefix) and 'results' not in filename]
all_part_paths = [datafolder + filename for filename in all_part_files]

# Creating Trial Dataframe

In [ ]:
# Each participant has their own file, generated from the Heroku postgres output in the 
# preprocessing notebook

part_stats_dicts = []
scarcity_level = 0.25
for file in all_part_paths:
    with open(file, 'r') as f:
        participant = json.load(f)
    try:
        beginhit = datetime.datetime.strptime(participant['Beginhit'], '%Y-%m-%d %H:%M:%S.%f')
    except TypeError:
        beginhit = None
    try:
        endhit = datetime.datetime.strptime(participant['Endhit'], '%Y-%m-%d %H:%M:%S.%f')
    except TypeError:
        endhit = None
    try:    
        beginexp = datetime.datetime.strptime(participant['Beginexp'], '%Y-%m-%d %H:%M:%S.%f')
    except TypeError:
        beginexp = None
    
    try:
        hitLength = (endhit-beginhit).seconds
    except TypeError:
        hitLength = None
    
    try:
        expLength = (endhit-beginexp).seconds
    except TypeError:
        expLength = None
        
    numPracTrials = 0
    trialLengths = []
    trialScores = []
    expectedScores = []
    numClicks = []
    avgClickLevel = []
    rewardsWithheld = []
    demographicsAvailable = False
    effort = "-1"
    age = None
    gender = None
    colorblind = None
    
    print(participant['workerId'])
    try:
        firstTrialStamp = participant['data'][0]['dateTime']
    except:
        print("No data")
        continue
    lastTrialStamp = participant['data'][-1]['dateTime']
    if hitLength is None:
        hitLength = (lastTrialStamp - firstTrialStamp) / 1000
    if expLength is None:
        expLength = (lastTrialStamp - firstTrialStamp) / 1000
        
    # Dictionary to check whether stroop tasks have been completed
    stroop_timestamps = {
        "1" : {
            "start" : 0,
            "end" : 0,
            "done" : False
        },
        "2" : {
            "start" : 0,
            "end" : 0,
            "done" : False
        }
    }
    # Dictionary to check whether mouselab task has been completed
    mdp_timestamps = {
        "start" : 0,
        "end" : 0,
        "done": False
    }
    
    # Use state machine to parse participant trial data, update state 
    #  based on trials seen
    current_state = "stroop_1"
    stroop1Completed = 0
    stroop1Correct = 0
    stroop2Completed = 0
    stroop2Correct = 0
    last_trial = participant['data'][0]
    feedback = ""
    comments = ""
    for idx, trial in enumerate(participant['data']):
        trial_type = trial['trialdata']['trial_type']
        try:
            trial_id = str(trial['trialdata']['trial_id'])
        except KeyError:
            trial_id = ""
        
        # Start 1st stroop task
        if trial_id == "stroop_1_ready_1" and not stroop_timestamps["1"]["done"]:
            stroop_timestamps["1"]["start"] = trial['dateTime']
        
        # Start 2nd stroop task
        if trial_id == "stroop_2_ready_1" and not stroop_timestamps["2"]["done"]:
            stroop_timestamps["2"]["start"] = trial['dateTime']
        
        # Finish 1st stroop task
        if trial_id == "finish_distractor_1" and not stroop_timestamps["1"]["done"]:
            stroop_timestamps["1"]["end"] = trial['dateTime']
            stroop_timestamps["1"]["done"] = True
        
        # Finish 2nd stroop task
        if trial_id == "finish_distractor_2" and not stroop_timestamps["2"]["done"]:
            stroop_timestamps["2"]["end"] = trial['dateTime']
            stroop_timestamps["2"]["done"] = True
        
        # Start mouselab trials
        if trial_id == "mouselab_instructions_1" and not mdp_timestamps["done"]:
            mdp_timestamps["done"] = True
            mdp_timestamps["start"] = trial['dateTime']
        
        # Finish mouselab trials
        if trial_id.startswith("final_quiz"):
            mdp_timestamps["end"] = trial['dateTime']
            
        # Stroop trial encountered, count whether it belongs to the first set or second set
        if "congruent" in trial_id or "incongruent" in trial_id or "unrelated" in trial_id:
            if not stroop_timestamps["1"]["done"]:
                stroop1Completed += 1
                
                if trial['trialdata']["response"].lower() == trial['trialdata']["correct_response"].lower():
                    stroop1Correct += 1
                
            else:
                stroop2Completed += 1
                if trial['trialdata']["response"].lower() == trial['trialdata']["correct_response"].lower():
                    stroop2Correct += 1
        
        # Mouselab trial encountered
        if trial_type == 'mouselab-mdp':
            if trial_id.startswith('practice'):
                numPracTrials += 1
            else:
                trialLengths.append(trial['trialdata']['trialTime'])
                
                trialScores.append(trial['trialdata']['score'])
                rewardsWithheld.append(trial['trialdata']['withholdReward'])
                
                # Getting expected scores
                g_truth = [0.0] + trial['trialdata']['stateRewards'][1:]
                mEnv = MouselabEnv.new_symmetric_registered('high_increasing', ground_truth=g_truth)
                clicks = trial['trialdata']['queries']['click']['state']['target']
                for click in clicks:
                    mEnv.step(int(click))
                
                
                # Saving the number of clicks and the average depth of all clicks
                numClicks.append(len(clicks))
                
                # Level one nodes are 1,5,9
                clicksL1 = len([c for c in clicks if int(c) in [1,5,9]])
                clicksL2 = len([c for c in clicks if int(c) in [2,6,10]])
                clicksL3 = len([c for c in clicks if int(c) in [3,4,7,8,11,12]])
                try:
                    avgLevel = (clicksL1 + 2*clicksL2 + 3*clicksL3)/len(clicks)
                except:
                    avgLevel = 0
                    
                avgClickLevel.append(avgLevel)
                
                # plan quality is expected score of the trial minus costs
                planQuality = mEnv._term_reward() + trial['trialdata']['costs']/(1.0 if participant['condition'] == 0 else scarcity_level)
                expectedScores.append(planQuality)
                
        # Save information about end questionnaire
        if trial_type == 'survey-html-form':
            if 'effort' in trial['trialdata']['response']:
                effort = trial['trialdata']['response']['effort']
                demographicsAvailable = True
        
        # Participant failed quiz
        if trial_id.startswith("finish_fail"):
            demographicsAvailable = True
        
        # Save information about participant feedback/comments
        if trial_type == 'survey-text':
            feedback = trial['trialdata']['response']['Q0']
            comments = trial['trialdata']['response']['Q2']
        last_trial = trial

    # Check whether participant dropped out by seeing whether any of the phases doesn't have an end timestamp
    dropoutPoint = None
    for obj, type_ in zip([mdp_timestamps, stroop_timestamps["1"], stroop_timestamps["2"]], ["mdp", "s1", "s2"]):
        if obj["start"] > 0 and obj["end"] == 0:
            dropoutPoint = type_
            obj["end"] = lastTrialStamp
            
    # Save participant statistics in dictionary   
    part_stats_dict = {
        "workerId": participant['workerId'],
        "Beginhit": beginhit,
        "Endhit": endhit,
        "Beginexp": beginexp,
        "psiturkStatus" : participant["psiturkStatus"],
        "browser" : participant["browser"],
        "platform" : participant["platform"],
        "language" : participant["language"],
        "hitLength": hitLength,
        "expLength": expLength,
        "totalLengthSum": (lastTrialStamp - firstTrialStamp) / 1000,
        "numQuizAttempts": numPracTrials / 2,
        "trialLengths": trialLengths,
        "trialScores" : trialScores,
        "numTrialsCompleted": len(trialLengths),
        "rewardsWithheld": rewardsWithheld,
        "effort": effort,
        "condition": participant['condition'],
        "stroop1Length" : (stroop_timestamps["1"]["end"] - stroop_timestamps["1"]["start"]) / 1000,
        "stroop2Length" : (stroop_timestamps["2"]["end"] - stroop_timestamps["2"]["start"]) / 1000,
        "mouselabLength" : (mdp_timestamps["end"] - mdp_timestamps["start"]) / 1000,
        "stroop1Completed" : stroop1Completed,
        "stroop2Completed" : stroop2Completed,
        "stroop1Correct" : stroop1Correct,
        "stroop2Correct" : stroop2Correct,
        "dropoutPoint" : dropoutPoint,
        "feedback": feedback,
        "comments": comments,
        "expectedScores": expectedScores,
        "numClicks": numClicks,
        "avgClickLevel" : avgClickLevel,
        "demographicsAvailable": demographicsAvailable
    }
    
    # Get final score from data file, or if not present, sum up the scores of all the rewarded trials
    if 'questiondata' in participant and 'final_score' in participant['questiondata']:
        part_stats_dict['finalScore'] = participant['questiondata']['final_score']
    else:
        part_stats_dict['finalScore'] = sum([score for (score, withheld) in zip(trialScores, rewardsWithheld) if not withheld])
    
    part_stats_dict["expLengthSum"] = part_stats_dict["stroop1Length"] + part_stats_dict["mouselabLength"] + part_stats_dict["stroop2Length"]
    part_stats_dict["expLengthDiff"] = part_stats_dict["expLengthSum"] - part_stats_dict["expLength"]
    if len(trialLengths) > 0:
        part_stats_dict["averageTrialLength"] = sum(trialLengths) / len(trialLengths)
        part_stats_dict["propWithheld"] = sum(rewardsWithheld) / len(rewardsWithheld)
        
    part_stats_dicts.append(part_stats_dict)
    

In [ ]:
# Compiling all data into trial data

all_trial_data = {
    "trialScores": [],
    "expectedScores": [],
    "numClicks" : [],
    "avgClickLevel" : [],
    "trialNumbers": [],
    "scarce": [],
    "numRewardedTrials": [],
    "numUnrewardedTrials": [],
    "workerId": [],
    "rewardsWithheld" : [],
}

count = 0;

for part in part_stats_dicts:
    if len(part['trialScores']) not in [30, 120]:
        print(part['condition'], part['workerId'], "MDP Data Incomplete")
        continue
        
    # Include in analysis only those participants whose data is complete
    if not part['demographicsAvailable']:
        print(part['condition'], part['workerId'], "Demographics missing")
        continue
    count += 1
    all_trial_data['trialScores'] += part['trialScores']
    all_trial_data['rewardsWithheld'] += part['rewardsWithheld']
    all_trial_data['workerId'] += [part['workerId']] * len(part['trialScores'])
    all_trial_data['expectedScores'] += part['expectedScores']
    all_trial_data['numClicks'] += part['numClicks']
    all_trial_data['avgClickLevel'] += part['avgClickLevel']
    all_trial_data['trialNumbers'] += list(range(1,len(part['trialScores'])+1))
    all_trial_data['scarce'] += [int(len(part['trialScores']) == 120)] * len(part['trialScores'])
    all_trial_data['numRewardedTrials'] += [i - sum(part['rewardsWithheld'][0:i]) for i in range(len(part['trialScores']))]
    all_trial_data['numUnrewardedTrials'] += [sum(part['rewardsWithheld'][0:i]) for i in range(len(part['trialScores']))]

all_trials_df = pd.DataFrame(all_trial_data)


In [ ]:
all_trials_df

## Analyzing Inferred Strategies

The strategies used by the participants on each trial were inferred by the Computational Microscope of the mcl_toolbox repository based on the click sequences made by the participants. These are then read in this section and added to the dataframe.

(The results from this analysis were not used in the paper, so this data was not made available. Skip this section.)

In [ ]:
# Defining paths

# CM inferred strategies
results_path = '../results'
inferred_path = results_path + "/cm/inferred_strategies"

scarce_inferred = inferred_path + "/scarcity_scarce/strategies.pkl"
control_inferred = inferred_path + "/scarcity_control/strategies.pkl"

# Output files for strategies
scarce_file = results_path + "/mouselab-mdp-final-scarce.csv"
control_file = results_path + "/mouselab-mdp-final-control.csv"

# Expected scores of all strategies
strategy_scores_scarce_path = results_path + "/cm/strategy_scores/scarcity_scarce_clickcost_0.25_strategy_scores.pkl"
strategy_scores_control_path = results_path + "/cm/strategy_scores/scarcity_control_clickcost_1.0_strategy_scores.pkl"


scarce_df_rows = []
control_df_rows = []

In [ ]:
# Reading csv files

with open(scarce_file, newline='') as file:
    reader = csv.reader(file)
    for idx, row in enumerate(reader):
        if idx == 0: continue
        scarce_df_rows.append([row[0], int(row[1])])
        
with open(control_file, newline='') as file:
    reader = csv.reader(file)
    for idx, row in enumerate(reader):
        if idx == 0: continue
        control_df_rows.append([row[0], int(row[1])])

In [ ]:
# Reading inferred strategies

with open(scarce_inferred, 'rb') as f:
    scarce_strategies = pickle.load(f)
    
with open(control_inferred, 'rb') as f:
    control_strategies = pickle.load(f)

In [ ]:
strategy_df = all_trials_df[['workerId', 'trialNumbers', 'scarce']].copy()
strategy_df['strategy'] = [None] * len(strategy_df) 

In [ ]:
for workerId, stratList in scarce_strategies.items():
    try:
        strategy_df.loc[strategy_df.workerId == workerId, 'strategy'] = stratList
    except:
        continue
    
for workerId, stratList in control_strategies.items():
    try:
        strategy_df.loc[strategy_df.workerId == workerId, 'strategy'] = stratList
    except:
        continue

In [ ]:
# Reading strategy scores

with open(strategy_scores_scarce_path, 'rb') as file:
    strategy_scores_scarce = pickle.load(file)
    
with open(strategy_scores_control_path, 'rb') as file:
    strategy_scores_control = pickle.load(file)


In [ ]:
# Creating the strategy score dataframes

strategy_scores_scarce_list = [[k,v] for k,v in strategy_scores_scarce.items()]
strategy_scores_control_list = [[k,v] for k,v in strategy_scores_control.items()]

scarce_cluster_df = pd.DataFrame(strategy_scores_scarce_list, columns=["strategy", "score"])
control_cluster_df = pd.DataFrame(strategy_scores_control_list, columns=["strategy", "score"])

# Scale to 0-1:
scarce_cluster_df['scoreScaled'] = (scarce_cluster_df['score'] - scarce_cluster_df['score'].min()) / (scarce_cluster_df['score'].max() - scarce_cluster_df['score'].min())
control_cluster_df['scoreScaled'] = (control_cluster_df['score'] - control_cluster_df['score'].min()) / (control_cluster_df['score'].max() - control_cluster_df['score'].min())



In [ ]:
# Cluster the strategy scores

scarce_clusters = scipy.cluster.vq.kmeans(scarce_cluster_df['scoreScaled'], k_or_guess=[0.6,0.5,0.4])
scarce_cluster_centers = scarce_clusters[0]

control_clusters = scipy.cluster.vq.kmeans(control_cluster_df['scoreScaled'], k_or_guess=[0.6,0.5,0.4])
control_cluster_centers = control_clusters[0]

In [ ]:
# Compute the cluster labels: 0 - adaptive, 1 - moderately adaptive, 2 - maladaptive

scarce_clusters_repeated = np.repeat(np.expand_dims(scarce_cluster_centers, axis=0), len(scarce_cluster_df), axis=0)
scarce_cluster_distances = (scarce_clusters_repeated - np.repeat(np.expand_dims(np.array(scarce_cluster_df['scoreScaled']), axis=1),repeats=len(scarce_cluster_centers), axis=1))**2
scarce_cluster_labels = np.argmin(scarce_cluster_distances, axis=1)

control_clusters_repeated = np.repeat(np.expand_dims(control_cluster_centers, axis=0), len(control_cluster_df), axis=0)
control_cluster_distances = (control_clusters_repeated - np.repeat(np.expand_dims(np.array(control_cluster_df['scoreScaled']), axis=1),repeats=len(control_cluster_centers), axis=1))**2
control_cluster_labels = np.argmin(control_cluster_distances, axis=1)

In [ ]:
# Create dictionary of cluster labels:

scarce_cluster_dict = { strat: cluster for (strat, cluster) in zip(scarce_cluster_df.strategy, scarce_cluster_labels)}

control_cluster_dict = { strat: cluster for (strat, cluster) in zip(control_cluster_df.strategy, control_cluster_labels)}


In [ ]:
# Add clusters and strategy scores to strategy_df 

strategy_df['cluster'] = [None] * len(strategy_df)
strategy_df['strategyScores'] = [None] * len(strategy_df)

for idx, row in strategy_df.iterrows():
    if row['scarce'] == 0:
        strategy_df.at[idx, 'cluster'] = scarce_cluster_dict[row['strategy']-1]
        strategy_df.at[idx, 'strategyScores'] = strategy_scores_scarce[row['strategy']-1] / scarcity_level
    else:
        strategy_df.at[idx, 'cluster'] = control_cluster_dict[row['strategy']-1]
        strategy_df.at[idx, 'strategyScores'] = strategy_scores_control[row['strategy']-1]
    
    
strategy_df['strategyScores'] = strategy_df['strategyScores'].astype("float")

In [ ]:
# Merging trial data with strategy data

strategy_cols = ["workerId", "trialNumbers", "strategy", "cluster", "strategyScores"]

all_trials_df = pd.merge(all_trials_df, strategy_df[strategy_cols], on=['workerId', "trialNumbers"])


In [ ]:
all_trials_df

## Removing Outliers

In [ ]:
# Group statistics by participant
all_part_groups = all_trials_df.groupby('workerId')
control_part_groups = all_trials_df.loc[all_trials_df.scarce==0].groupby("workerId")
scarce_part_groups = all_trials_df.loc[all_trials_df.scarce==1].groupby("workerId")

# Get the averages of each statistic by participant
control_part_averages = control_part_groups.mean()
scarce_part_averages = scarce_part_groups.mean()
all_part_averages = all_part_groups.mean()

# Compute the proportion of trials with no clicks
all_part_averages['pctgSomeClick'] = (all_part_groups.apply(lambda x: x[x > 0].count()) / all_part_groups.count())['numClicks']
scarce_part_averages['pctgSomeClick'] = (scarce_part_groups.apply(lambda x: x[x > 0].count()) / scarce_part_groups.count())['numClicks']
control_part_averages['pctgSomeClick'] = (control_part_groups.apply(lambda x: x[x > 0].count()) / control_part_groups.count())['numClicks']

### Removing Based on Selected Criteria

In [ ]:
selected_exclusion_col = 'pctgSomeClick'

In [ ]:
# Removing outliers 

# Excluding those that have 100% of their trials with no clicks


zscore_threshold = (0.0 - all_part_averages[selected_exclusion_col].mean()) / all_part_averages[selected_exclusion_col].std()
excluded_parts_control = control_part_averages[control_part_averages[selected_exclusion_col] == 0]
included_parts_control = control_part_averages[control_part_averages[selected_exclusion_col] > 0]

zscore_threshold = (0.0 - scarce_part_averages[selected_exclusion_col].mean()) / scarce_part_averages[selected_exclusion_col].std()
excluded_parts_scarce = scarce_part_averages[scarce_part_averages[selected_exclusion_col] == 0]
included_parts_scarce = scarce_part_averages[scarce_part_averages[selected_exclusion_col] > 0]

num_excluded_parts = len(excluded_parts_scarce) + len(excluded_parts_control)

print("Number of participants excluded: {0} ({1:0.2f}%)".format(
    num_excluded_parts,
    100 * num_excluded_parts /len(all_trials_df.groupby("workerId"))
))
print("\tControl: {0} ({1:0.2f}%)".format(
    len(excluded_parts_control),
    100 * len(excluded_parts_control) / len(control_part_averages)
))
print("\tScarce: {0} ({1:0.2f}%)".format(
    len(excluded_parts_scarce),
    100 * len(excluded_parts_scarce) / len(scarce_part_averages)
))
included_part_IDs = list(included_parts_scarce.index) + list(included_parts_control.index)
outliers_excluded_part = all_trials_df.loc[all_trials_df.workerId.isin(included_part_IDs)].reset_index(drop=True)
outliers_excluded_part['trialNumbers'] = outliers_excluded_part['trialNumbers'].astype('int64')
outliers_excluded_part['scarce'] = outliers_excluded_part['scarce'].astype('int64')
outliers_excluded_part['numRewardedTrials'] = outliers_excluded_part['numRewardedTrials'].astype('int64')
outliers_excluded_part['numUnrewardedTrials'] = outliers_excluded_part['numUnrewardedTrials'].astype('int64')


print("Number of total trials excluded: {0} ({1:0.2f}%)".format(len(all_trials_df)-len(outliers_excluded_part), 100-100*len(outliers_excluded_part)/(len(all_trials_df))))


In [ ]:
# Data to use for the remainder of the analysis

model_save_folder = "../results/model_sim_data/"
control_model = "2.0.3.0.0"
scarce_model = "2.0.3.0.0"
num_simulations = 1
model_file_name = f"c{control_model}_s{scarce_model}_{num_simulations}.csv"

use_model_data = False

if use_model_data:
    filtered_data = pd.read_csv(model_save_folder + model_file_name)
else:
    filtered_data = all_trials_df

In [ ]:
filtered_data

# Visualizing Data

## Expected Scores

Only first graph reported in paper

In [ ]:
# Plotting mean expected score vs. trial number for each condition

data_to_copy = filtered_data.copy() # all_trials_df, outliers_excluded, or outliers_excluded_part

num_scarce = len(data_to_copy.loc[data_to_copy.scarce == 1].groupby('workerId'))
num_control = len(data_to_copy.loc[data_to_copy.scarce == 0].groupby('workerId'))

# Taking the mean expected score over all trials 
scarce_data_all = np.array(data_to_copy.loc[data_to_copy.scarce == 1].groupby('trialNumbers').mean()["expectedScores"])
scarce_data_all_se = np.array(data_to_copy.loc[data_to_copy.scarce == 1].groupby('trialNumbers').std()["expectedScores"]) / np.sqrt(num_scarce)
control_data = np.array(data_to_copy.loc[data_to_copy.scarce == 0].groupby('trialNumbers').mean()["expectedScores"])
control_data_se = np.array(data_to_copy.loc[data_to_copy.scarce == 0].groupby('trialNumbers').std()["expectedScores"]) / np.sqrt(num_scarce)
scarce_data_first_30 = scarce_data_all[0:30]
scarce_data_first_30_se = scarce_data_all_se[0:30]

# Taking only the rewarded trials in the scarce condition
rewarded_trials_df = pd.DataFrame(columns=data_to_copy.columns)
for worker in np.unique(data_to_copy.workerId):
    workerRows = data_to_copy.loc[(data_to_copy.scarce == 1) & (data_to_copy.workerId == worker)].reset_index(drop=True)
    rewarded = workerRows.loc[workerRows.rewardsWithheld == False]
    rewarded['trialNumbers'] = list(range(1,len(rewarded)+1))
    rewarded_trials_df = pd.concat([rewarded_trials_df,rewarded], ignore_index=True)

rewarded_trials_df['trialNumbers'] = rewarded_trials_df['trialNumbers'].astype("int64")
rewarded_trials_df['scarce'] = rewarded_trials_df['scarce'].astype("int64")
rewarded_trials_df['numRewardedTrials'] = rewarded_trials_df['numRewardedTrials'].astype("int64")
rewarded_trials_df['numUnrewardedTrials'] = rewarded_trials_df['numUnrewardedTrials'].astype("int64")
rewarded_trials_df['rewardsWithheld'] = rewarded_trials_df['rewardsWithheld'].astype("boolean")
scarce_data_rewarded = np.array(rewarded_trials_df.groupby('trialNumbers').mean()["expectedScores"])
scarce_data_rewarded_sd = np.array(rewarded_trials_df.groupby('trialNumbers').std()["expectedScores"])

plt.figure()
#plt.title("Learning Curves over All Trials")
plt.plot(list(range(len(scarce_data_all))), scarce_data_all, label="Scarcity", color='darkturquoise')
plt.fill_between(list(range(len(scarce_data_all))), scarce_data_all + 1.96*scarce_data_all_se, scarce_data_all - 1.96*scarce_data_all_se, alpha=0.15, color='darkturquoise')
plt.plot(list(range(len(control_data))), control_data, label="Control", color='firebrick')
plt.fill_between(list(range(len(control_data))), control_data + 1.96*control_data_se, control_data - 1.96*control_data_se, alpha=0.15, color='firebrick')
plt.axvline(13, linestyle='--', color='darkturquoise')
plt.axvline(9, linestyle='--', color='firebrick')
#plt.axhline(39.97, color='k', label='Optimal', linestyle="--")
plt.legend(loc="lower right",prop={'size': 12})
#plt.ylim([10,60])
plt.xlim([0, 125])
plt.xlabel("Trial Number",fontsize=12)
plt.ylabel("Average expected trial score",fontsize=12)
plt.show()

plt.figure()
plt.title("Learning Curves over Rewarded Trials")
plt.plot(list(range(len(scarce_data_rewarded))), scarce_data_rewarded, label="Scarce")
plt.plot(list(range(len(control_data))), control_data, label="Control")
plt.legend()
plt.ylim([10,60])
plt.xlim([0, 40])
plt.xlabel("Trial Number")
plt.ylabel("Average expected score")
plt.show()

plt.figure()
plt.title("Learning Curves over First 30 trials")
plt.plot(list(range(len(scarce_data_first_30))), scarce_data_first_30, label="Scarce")
plt.plot(list(range(len(control_data))), control_data, label="Control")
plt.legend()
plt.ylim([10,60])
plt.xlim([0, 40])
plt.xlabel("Trial Number")
plt.ylabel("Average expected score")
plt.show()



## Clicks

Not reported in publication

In [ ]:
# Plotting mean expected score vs. trial number for each condition

data_to_copy = filtered_data.copy() # all_trials_df, outliers_excluded, or outliers_excluded_part

# Taking the mean expected score over all trials 
scarce_level_all = np.array(data_to_copy.loc[data_to_copy.scarce == 1].groupby('trialNumbers').mean()["avgClickLevel"])
scarce_level_all_sd = np.array(data_to_copy.loc[data_to_copy.scarce == 1].groupby('trialNumbers').std()["avgClickLevel"])
control_level_all = np.array(data_to_copy.loc[data_to_copy.scarce == 0].groupby('trialNumbers').mean()["avgClickLevel"])
control_level_sd = np.array(data_to_copy.loc[data_to_copy.scarce == 0].groupby('trialNumbers').std()["avgClickLevel"])

# Taking the mean expected score over all trials 
scarce_num_all = np.array(data_to_copy.loc[data_to_copy.scarce == 1].groupby('trialNumbers').mean()["numClicks"])
scarce_num_all_sd = np.array(data_to_copy.loc[data_to_copy.scarce == 1].groupby('trialNumbers').std()["numClicks"])
control_num_all = np.array(data_to_copy.loc[data_to_copy.scarce == 0].groupby('trialNumbers').mean()["numClicks"])
control_num_sd = np.array(data_to_copy.loc[data_to_copy.scarce == 0].groupby('trialNumbers').std()["numClicks"])

plt.figure()
plt.title("Learning Curves of Number of Clicks")
plt.plot(list(range(len(scarce_num_all))), scarce_num_all, label="Scarce")
plt.fill_between(list(range(len(scarce_num_all))), scarce_num_all + scarce_num_all_sd, scarce_num_all - scarce_num_all_sd, alpha=0.1)
plt.plot(list(range(len(control_num_all))), control_num_all, label="Control")
plt.fill_between(list(range(len(control_num_all))), control_num_all + control_num_sd, control_num_all - control_num_sd, alpha=0.1)
plt.axhline(3.56, color='k', label='Optimal', linestyle="--")
plt.legend()
plt.xlim([0, 130])
plt.xlabel("Trial Number")
plt.ylabel("Average number of clicks")
plt.show()

plt.figure()
plt.title("Learning Curves of Average Click Level")
plt.plot(list(range(len(scarce_level_all))), scarce_level_all, label="Scarce")
plt.fill_between(list(range(len(scarce_level_all))), scarce_level_all + scarce_level_all_sd, scarce_level_all - scarce_level_all_sd, alpha=0.1)
plt.plot(list(range(len(control_level_all))), control_level_all, label="Control")
plt.fill_between(list(range(len(control_level_all))), control_level_all + control_level_sd, control_level_all - control_level_sd, alpha=0.1)
plt.axhline(2.968, color='k', label='Optimal', linestyle="--")
plt.legend()
plt.xlim([0, 130])
plt.xlabel("Trial Number")
plt.ylabel("Average click level")
plt.show()




## Strategy Scores

Not reported in publication

In [ ]:
# Plotting mean expected score vs. trial number for each condition

data_to_copy = filtered_data.copy() # all_trials_df, outliers_excluded, or outliers_excluded_part

scarce_trial_group = data_to_copy.loc[data_to_copy.scarce == 1].groupby('trialNumbers')
control_trial_group = data_to_copy.loc[data_to_copy.scarce == 0].groupby('trialNumbers')

# Taking the mean expected score over all trials 
scarce_data_all = np.array(scarce_trial_group.mean()["strategyScores"])
scarce_data_all_sd = np.array(scarce_trial_group.std()["strategyScores"])
control_data = np.array(control_trial_group.mean()["strategyScores"])
control_data_sd = np.array(control_trial_group.std()["strategyScores"])

# Getting the proportions of clusters
scarce_data_adaptive = np.array(scarce_trial_group["cluster"].apply(lambda x: (x==0).sum()) / scarce_trial_group["cluster"].count())
control_data_adaptive = np.array(control_trial_group["cluster"].apply(lambda x: (x==0).sum()) / control_trial_group["cluster"].count())
scarce_data_modadaptive = np.array(scarce_trial_group["cluster"].apply(lambda x: (x==1).sum()) / scarce_trial_group["cluster"].count())
control_data_modadaptive = np.array(control_trial_group["cluster"].apply(lambda x: (x==1).sum()) / control_trial_group["cluster"].count())
scarce_data_maladaptive = np.array(scarce_trial_group["cluster"].apply(lambda x: (x==2).sum()) / scarce_trial_group["cluster"].count())
control_data_maladaptive = np.array(control_trial_group["cluster"].apply(lambda x: (x==2).sum()) / control_trial_group["cluster"].count())
#control_data = np.array(data_to_copy.loc[data_to_copy.scarce == 0].groupby('trialNumbers').mean()["strategyScores"])
#control_data_sd = np.array(data_to_copy.loc[data_to_copy.scarce == 0].groupby('trialNumbers').std()["strategyScores"])

# Taking only the rewarded trials in the scarce condition
rewarded_trials_df = pd.DataFrame(columns=data_to_copy.columns)
for worker in np.unique(data_to_copy.workerId):
    workerRows = data_to_copy.loc[(data_to_copy.scarce == 1) & (data_to_copy.workerId == worker)].reset_index(drop=True)
    rewarded = workerRows.loc[workerRows.rewardsWithheld == False]
    rewarded['trialNumbers'] = list(range(1,len(rewarded)+1))
    rewarded_trials_df = pd.concat([rewarded_trials_df,rewarded], ignore_index=True)

rewarded_trials_df['trialNumbers'] = rewarded_trials_df['trialNumbers'].astype("int64")
rewarded_trials_df['scarce'] = rewarded_trials_df['scarce'].astype("int64")
rewarded_trials_df['numRewardedTrials'] = rewarded_trials_df['numRewardedTrials'].astype("int64")
rewarded_trials_df['numUnrewardedTrials'] = rewarded_trials_df['numUnrewardedTrials'].astype("int64")
rewarded_trials_df['rewardsWithheld'] = rewarded_trials_df['rewardsWithheld'].astype("boolean")
scarce_data_rewarded = np.array(rewarded_trials_df.groupby('trialNumbers').mean()["strategyScores"])
scarce_data_rewarded_sd = np.array(rewarded_trials_df.groupby('trialNumbers').std()["strategyScores"])

plt.figure()
plt.title("Strategy score Learning Curves over All Trials")
plt.plot(list(range(len(scarce_data_all))), scarce_data_all, label="Scarce")
plt.fill_between(list(range(len(scarce_data_all))), scarce_data_all + scarce_data_all_sd, scarce_data_all - scarce_data_all_sd, alpha=0.1)
plt.plot(list(range(len(control_data))), control_data, label="Control")
plt.fill_between(list(range(len(control_data))), control_data + control_data_sd, control_data - control_data_sd, alpha=0.1)
plt.axhline(np.max(np.array(strategy_scores_control_list)[:,1], axis=0), color='k', label='Optimal', linestyle="--")
plt.legend()
plt.xlim([0, 130])
plt.xlabel("Trial Number")
plt.ylabel("Average strategy score")
plt.show()

plt.figure()
plt.title("Learning Curves over Rewarded Trials")
plt.plot(list(range(len(scarce_data_rewarded))), scarce_data_rewarded, label="Scarce")
plt.plot(list(range(len(control_data))), control_data, label="Control")
plt.legend()
plt.xlim([0, 40])
plt.xlabel("Trial Number")
plt.ylabel("Average strategy score")
plt.show()

plt.figure()
plt.title("Strategy Type Change")
alpha = 0.7
plt.plot(list(range(len(scarce_data_all))), scarce_data_adaptive, label="Scarce Adaptive", color='b',alpha=alpha)
plt.plot(list(range(len(control_data))), control_data_adaptive, label="Control Adaptive",color='orange',alpha=alpha)
plt.plot(list(range(len(scarce_data_all))), scarce_data_modadaptive, label="Scarce Mod-Adaptive", color='b',alpha=alpha, linestyle="--")
plt.plot(list(range(len(control_data))), control_data_modadaptive, label="Control Mod-Adaptive",color='orange',alpha=alpha, linestyle="--")
plt.plot(list(range(len(scarce_data_all))), scarce_data_maladaptive, label="Scarce Maldaptive", color='b',alpha=alpha, linestyle=":")
plt.plot(list(range(len(control_data))), control_data_maladaptive, label="Control Maldaptive",color='orange',alpha=alpha, linestyle=":")
plt.legend()
plt.ylim([0,1])
plt.xlim([0, 130])
plt.xlabel("Trial Number")
plt.ylabel("Proportion of Trials")
plt.show()


# 0.1 - Transforming Response Data

Not used for results reported in paper

In [ ]:
def scale_to_range(arr, lb, ub):
    return ((arr - arr.min()) / (arr.max() - arr.min())) * (ub - lb) + lb

In [ ]:
# Selecting the response variable

response_vars = ['expectedScores', 'avgClickLevel', 'strategyScores']

In [ ]:
# Transforming the data

data_to_copy = filtered_data.copy() # or outliers_excluded
transformed_trials_df = data_to_copy.copy()
trans_list = {res: [] for res in response_vars}

for response_var in response_vars:
    

    log_transformed_trials_df = data_to_copy.copy()
    sq_transformed_trials_df = data_to_copy.copy()
    sq2_transformed_trials_df = data_to_copy.copy()
    cb_transformed_trials_df = data_to_copy.copy()
    cb2_transformed_trials_df = data_to_copy.copy()
    cb3_transformed_trials_df = data_to_copy.copy()
    arcsin_transformed_trials_df = data_to_copy.copy()
    bc_transformed_trials_df = data_to_copy.copy()
    yj_transformed_trials_df = data_to_copy.copy()
    inverse_transformed_trials_df = data_to_copy.copy()

    # Compute separate dataframes for each transformation
    log_transformed_trials_df[response_var] = np.log(transformed_trials_df[response_var].max() + 1 - transformed_trials_df[response_var])
    sq_transformed_trials_df[response_var] = np.sqrt(transformed_trials_df[response_var].max() + 1 - transformed_trials_df[response_var])
    sq2_transformed_trials_df[response_var] = np.sqrt(-transformed_trials_df[response_var].min() + 1 + transformed_trials_df[response_var])
    cb_transformed_trials_df[response_var] = np.cbrt(transformed_trials_df[response_var])
    cb2_transformed_trials_df[response_var] = np.cbrt(transformed_trials_df[response_var].max() + 1 - transformed_trials_df[response_var])
    cb3_transformed_trials_df[response_var] = np.cbrt(-transformed_trials_df[response_var].min() + 1 + transformed_trials_df[response_var])
    arcsin_transformed_trials_df[response_var] = np.arcsin(np.sqrt(scale_to_range(transformed_trials_df[response_var], 0, 1)))
    bc_transformed_trials_df[response_var],lam = scipy.stats.boxcox(transformed_trials_df[response_var].max() + 1 - transformed_trials_df[response_var])
    #logit_transformed_trials_df[response_var] = scipy.special.logit(scale_to_range(transformed_trials_df[response_var], 0, 1))
    yj_transformed_trials_df[response_var],lam_y = scipy.stats.yeojohnson(transformed_trials_df[response_var])
    inverse_transformed_trials_df[response_var] = 1/(transformed_trials_df[response_var].max() + 1 - transformed_trials_df[response_var])

    # All the transformed data and the corresponding transformation
    trans_list[response_var] = [
        (transformed_trials_df, "Original data"),
        (log_transformed_trials_df, "Log(max(Y) - Y + 1)"),
        (sq_transformed_trials_df, "Sqrt(max(Y) - Y + 1)"),
        (sq2_transformed_trials_df, "Sqrt(-min(Y) + Y + 1)"),
        (cb_transformed_trials_df, "Cbrt(Y)"),
        (bc_transformed_trials_df, "Box-Cox(max(Y) - Y + 1), l={0:0.4f}".format(lam)),
        (cb2_transformed_trials_df, "Cbrt(max(Y) - Y + 1)"),
        (cb3_transformed_trials_df, "Cbrt(-min(Y) + Y + 1)"),
        (arcsin_transformed_trials_df, "Arcsin(sqrt(Y))"),
        (yj_transformed_trials_df, "Yeo-Johnson(Y), l={0:0.4f}".format(lam_y)),
        (inverse_transformed_trials_df, "1/(max(Y) - Y + 1)")
    ]
    
    # Plot all the transformed data
    fig, ax = plt.subplots(nrows=math.ceil(len(trans_list[response_var])/2),ncols=2,figsize=(10, 3 * math.ceil(len(trans_list[response_var])/2)),squeeze=False)
    plt.suptitle("Distributions of Transformed Data - {}".format(response_var))
    ax = ax.flatten()

    for idx, trans in enumerate(trans_list[response_var]):
        data = trans[0][response_var]
        label = trans[1]
        skew = scipy.stats.skew(data)
        density = scipy.stats.gaussian_kde(data)
        kurtosis = scipy.stats.kurtosis(data)
        n, x, _ = ax[idx].hist(data, density=True)
        ax[idx].set_title(label)
        ax[idx].plot(x, density(x))
        ax[idx].text(0.05,0.8, "S: {0:0.4f}\nK: {1:0.4f}".format(skew, kurtosis), transform=ax[idx].transAxes)

    plt.subplots_adjust(
                    wspace=0.4, 
                    hspace=0.4)
plt.show()

In [ ]:
# Plotting the response data

for response_var in response_vars:
    fig, ax = plt.subplots(nrows=math.ceil(len(trans_list[response_var])/2),ncols=2,figsize=(10, 3 * math.ceil(len(trans_list[response_var])/2)),squeeze=False)
    plt.suptitle("Transformed Response Data - {}".format(response_var))
    ax = ax.flatten()

    for idx, trans in enumerate(trans_list[response_var]):
        data = trans[0]
        scarce = data.loc[data.scarce == 1]
        control = data.loc[data.scarce == 0]
        label = trans[1]

        scarce_mean = np.array(scarce.groupby('trialNumbers').mean()[response_var])
        control_mean = np.array(control.groupby('trialNumbers').mean()[response_var])
        ax[idx].plot(list(range(len(scarce_mean))), scarce_mean, label="Scarce")
        ax[idx].plot(list(range(len(control_mean))), control_mean, label="Control")
        ax[idx].set_title(label)

    plt.subplots_adjust(
                    wspace=0.4, 
                    hspace=0.4)
plt.show()

## Linear Regression: Expected Score vs. Trial Number x Condition

In [ ]:
# formula = "expectedScores ~ trialNumbers + C(scarce) + trialNumbers:C(scarce)"

In [ ]:
# GLM Model 1 - Expected score vs. Trial number & Condition - all possible transformations

# Creating the list of data to subplot all residuals

all_residuals = {res: [] for res in response_vars}
mixed = True

for response_var in response_vars:
    formula = '{} ~ C(trialNumbers) + C(scarce)'.format(response_var);

    # Perform regression on each transformed dataset
    for idx, trans in enumerate(trans_list[response_var]):
        if mixed:
            glm = smf.mixedlm(formula=formula, data=trans[0], groups=trans[0]['workerId'])
        else:
            glm = smf.glm(formula=formula, data=trans[0])

        results = glm.fit()

        print("\n{} - {}:".format(response_var, trans[1]))
        print(results.summary())

        resids = results.resid_response if not mixed else results.resid
        normaltest = scipy.stats.normaltest(resids)
        
        # Save the results of the regression for plotting
        all_residuals[response_var].append((
            resids,
            trans[0][response_var],
            trans[1],
            normaltest
        ))


In [ ]:
# Plotting residuals for all GLM Models

analysis_cols = {res: [] for res in response_vars}
highest_normality_trans = {res: "" for res in response_vars}

for response_var in response_vars:
    fig, ax = plt.subplots(nrows=math.ceil(len(all_residuals[response_var])/2),ncols=2,figsize=(10, 3 * math.ceil(len(all_residuals[response_var])/2)),squeeze=False)
    plt.suptitle("Residuals of Transformed Data - {}".format(response_var))
    ax = ax.flatten()

    lowest_normality_stat = float("inf")
    for idx, trans in enumerate(all_residuals[response_var]):
        resids = trans[0]
        response_data = trans[1]
        label = trans[2]
        normaltest = trans[3]
        if normaltest.statistic < lowest_normality_stat:
            analysis_cols[response_var] = response_data
            highest_normality_trans[response_var] = label
            lowest_normality_stat = normaltest.statistic
        density = scipy.stats.gaussian_kde(resids)
        n, x, _ = ax[idx].hist(resids, density=True)
        ax[idx].set_title(label)
        ax[idx].plot(x, density(x))
        ax[idx].text(0.05,0.8, "S: {0:0.4f}\nP: {1:0.4f}".format(normaltest.statistic, normaltest.pvalue), transform=ax[idx].transAxes)

    plt.subplots_adjust(
                    wspace=0.4, 
                    hspace=0.4)
plt.show()

In [ ]:
# data which is to be used for the remainder of the analysis

# Displaying which of the data has the most normal residuals 
analysis_data = transformed_trials_df.copy()
for response_var in response_vars:
    print("{} - {}".format(response_var, highest_normality_trans[response_var]))
    analysis_data[response_var] = analysis_cols[response_var]
    

In [ ]:
# Visualize transformed data for each response variable

for response_var in response_vars:
    fig, ax = plt.subplots(nrows=1,ncols=2,figsize=(10, 3),squeeze=True)
    plt.suptitle("Transformed Response Data - {}".format(response_var),y=1.1)
    ax = ax.flatten()

    for idx, trans in enumerate([transformed_trials_df, analysis_data]):
        data = trans
        scarce = data.loc[data.scarce == 1]
        control = data.loc[data.scarce == 0]
        label = "Original" if idx == 0 else highest_normality_trans[response_var]

        scarce_mean = np.array(scarce.groupby('trialNumbers').mean()[response_var])
        control_mean = np.array(control.groupby('trialNumbers').mean()[response_var])
        ax[idx].plot(list(range(len(scarce_mean))), scarce_mean, label="Scarce")
        ax[idx].plot(list(range(len(control_mean))), control_mean, label="Control")
        ax[idx].set_title(label)

plt.show()

# 1.1.1 - Linear Regression: Finding Best Model of Learning vs. Performance

Used to compute partitions between fast learning phase and slow improvement phase for each condition

### Finding Best Model for Scarce Condition

In [ ]:


test_df = analysis_data

mixed = False
    
# For scarce trials
relevant_trials = test_df.loc[test_df.scarce == 1].copy()

all_trials_mean_scarce = relevant_trials.groupby("trialNumbers").mean()["expectedScores"]
all_trial_nos_scarce = list(range(1,len(all_trials_mean_scarce)+1))

sig_bic_scores_scarce = []
scarce_plot_data = []
scarce_glm_results = []

# Check all possible data splits for both conditions
for trialNum in range(2,60,1):
    isPerformanceTrial = [int(trial > trialNum) for trial in relevant_trials.trialNumbers]
    relevant_trials['isPerformanceTrial'] = isPerformanceTrial
    
    partition_1_df = relevant_trials.loc[relevant_trials.isPerformanceTrial == 0]
    partition_2_df = relevant_trials.loc[relevant_trials.isPerformanceTrial == 1]
    partition_1_mean_scores = partition_1_df.groupby("trialNumbers").mean()["expectedScores"]
    partition_2_mean_scores = partition_2_df.groupby("trialNumbers").mean()["expectedScores"]
    
    two_slope_formula = "expectedScores ~ trialNumbers + trialNumbers:C(isPerformanceTrial) + C(isPerformanceTrial)"
    one_slope_formula = "expectedScores ~ trialNumbers"
    
    # Fitting a single-slope model and a two-slope model to see which is better
    if mixed:
        glm_1 = smf.mixedlm(formula=two_slope_formula, data=relevant_trials, groups=relevant_trials['workerId'])
        glm_2 = smf.mixedlm(formula=one_slope_formula, data=relevant_trials, groups=relevant_trials['workerId'])
    else:
        glm_1 = smf.glm(formula=two_slope_formula, data=relevant_trials)
        glm_2 = smf.glm(formula=one_slope_formula, data=relevant_trials)

    try:
        results_1 = glm_1.fit()
        results_2 = glm_2.fit()
    except:
        continue
        
    bic1 = results_1.bic_llf
    bic2 = results_2.bic_llf
        
    # Saving the slopes and the single intercept of both phases from the two-slope model
    b1 = results_1.params['trialNumbers']
    b2 = results_1.params['trialNumbers'] + results_1.params['trialNumbers:C(isPerformanceTrial)[T.1]']
    i1 = results_1.params['Intercept']
    i2 = results_1.params['Intercept'] + results_1.params['C(isPerformanceTrial)[T.1]']
    
    b3 = results_2.params['trialNumbers']
    i3 = results_2.params['Intercept']
    
    if(bic1 < bic2):
        # Save data about significant partitions, where the bic of the two-slope model is higher than one-slope
        sig_bic_scores_scarce.append((trialNum, bic1))
        
    x1 = all_trial_nos_scarce[0:len(partition_1_mean_scores)]
    x2 = all_trial_nos_scarce[len(partition_1_mean_scores):len(partition_1_mean_scores)+len(partition_2_mean_scores)]
    scarce_plot_data.append((
        trialNum,
        x1,
        x2,
        partition_1_mean_scores,
        partition_2_mean_scores,
        b1,
        b2,
        i1,
        i2,
        b3,
        i3,
        partition_1_df,
        partition_2_df
    ))
    scarce_glm_results.append((trialNum, results_1, results_2))

In [ ]:
# Finding the best partition of all significant ones 
if use_model_data:
    partition_candidates = [13] # Use partition from human dataset
else:
    sig_bic_scores_scarce = np.array(sig_bic_scores_scarce)

    g_range = np.arange(0, 1, 0.0001)
    best_partitions = []

    for gamma in g_range:
        # Minimum bic
        partition_scores = sig_bic_scores_scarce[:,1]
        best_partition = sig_bic_scores_scarce[np.argmin(partition_scores),0]
        best_partitions.append(best_partition)

    partition_candidates = np.unique(best_partitions)



    print("Partition candidates: ")
    print(partition_candidates)
    plt.plot(g_range, best_partitions)
    plt.xlabel("Gamma (Decay Factor)")
    plt.ylabel("Best Partition")
    plt.show()


In [ ]:
# Showing GLM results - scarce
candidate_glm_results_scarce = [data for data in scarce_glm_results if float(data[0]) in partition_candidates]

for trialNum, results_1, results_2 in candidate_glm_results_scarce:
    print("Trial Num: {}".format(trialNum))
    print("\nTwo-Slope Model:\nBIC LLF: {0:0.4f}\n\nBIC Deviance: {1:0.4f}\n\nAIC: {2:0.4f}\n"
          .format(results_1.bic_llf,results_1.bic_deviance, results_1.aic))
    print(results_1.summary())
    
    print("\nOne-Slope Model:\nBIC LLF: {0:0.4f}\n\nBIC Deviance: {1:0.4f}\n\nAIC: {2:0.4f}\n"
          .format(results_2.bic_llf,results_2.bic_deviance, results_2.aic))
    print(results_2.summary())


In [ ]:
# Plotting candidate partitions - scarce
candidate_plot_data_scarce = [data for data in scarce_plot_data if float(data[0]) in partition_candidates]
#significant_plot_data_scarce = [data for data in scarce_plot_data if float(data[0]) in sig_bic_scores_scarce[:,0]]

which_plot = "candidates" #all, significant, candidates

if which_plot == "candidates":
    plot_data = candidate_plot_data_scarce
elif which_plot == "significant":
    plot_data = significant_plot_data_scarce
else:
    plot_data = scarce_plot_data
    
plot_single_slope = False
plot_two_slope = True
scatter = False
error = False

for trialNum, x1, x2, y1, y2, b1, b2, i1, i2,b3, i3, df_1, df_2 in plot_data:
    plt.figure()
    # Expected Score curves
    plt.plot(all_trial_nos_scarce, all_trials_mean_scarce, alpha=0.2, color='k')
    plt.plot(x1, y1, alpha=0.5)
    plt.plot(x2, y2, alpha=0.5)
    
    # Scatter points
    if scatter:
        plt.scatter(df_1['trialNumbers'], df_1['expectedScores'], color='b', marker='o', alpha=0.1,s=0.8)
        plt.scatter(df_2['trialNumbers'], df_2['expectedScores'], color='orange', marker='o', alpha=0.1,s=0.8)

    # Partition line
    plt.axvline(trialNum, linestyle="--", color='k',alpha=0.5)

    # Slope lines of one-slope model
    if plot_single_slope:
        x3 = np.concatenate([x1,x2],axis=0)
        plt.plot(x3, np.array(x3) * b3 + i3, color='g', alpha=0.5)
    
    if plot_two_slope:
        # Slope lines of two-slope model
        plt.plot(x1, np.array(x1) * b1 + i1, color='b')
        plt.plot(x2, np.array(x2) * b2 + i2, color='orange')
        
    if error:
        plt.fill_between(x1, scarce_data_all + scarce_data_all_sd, scarce_data_all - scarce_data_all_sd, alpha=0.1)
    
    plt.title("Trial: {}".format(trialNum))
    
plt.show()

In [ ]:
scarce_partition = partition_candidates[0] # Somehow choose based on possible candidates

# Separating into learning and performance phase
scarce_learning_data = analysis_data.loc[(analysis_data.scarce == 1) & (analysis_data.trialNumbers <= scarce_partition)]
scarce_performance_data = analysis_data.loc[(analysis_data.scarce ==1) & (analysis_data.trialNumbers > scarce_partition)]

# Without transformation
scarce_learning_data_orig = filtered_data.loc[(filtered_data.scarce == 1) & (filtered_data.trialNumbers <= scarce_partition)]
scarce_performance_data_orig = filtered_data.loc[(filtered_data.scarce ==1) & (filtered_data.trialNumbers > scarce_partition)]

# Store the slope, intercept, etc. of the best partition
scarce_partition_plot_data = [data for data in candidate_plot_data_scarce if data[0] == scarce_partition][0]

### Finding Best Model for Control Condition

In [ ]:
# GLM 3 - finding cut off point of learning phase

test_df = analysis_data

mixed = False
    
# For control trials
relevant_trials = test_df.loc[test_df.scarce == 0].copy()

all_trials_mean_control = relevant_trials.groupby("trialNumbers").mean()["expectedScores"]
all_trial_nos_control = list(range(1,len(all_trials_mean_control)+1))

sig_bic_scores_control = []
control_plot_data = []
control_glm_results = []
for trialNum in range(2,30,1):
    isPerformanceTrial = [int(trial > trialNum) for trial in relevant_trials.trialNumbers]
    relevant_trials['isPerformanceTrial'] = isPerformanceTrial
    
    partition_1_df = relevant_trials.loc[relevant_trials.isPerformanceTrial == 0]
    partition_2_df = relevant_trials.loc[relevant_trials.isPerformanceTrial == 1]
    partition_1_mean_scores = partition_1_df.groupby("trialNumbers").mean()["expectedScores"]
    partition_2_mean_scores = partition_2_df.groupby("trialNumbers").mean()["expectedScores"]
    
    two_slope_formula = "expectedScores ~ trialNumbers + C(scarce) + C(isPerformanceTrial) + trialNumbers:C(isPerformanceTrial)"
    one_slope_formula = "expectedScores ~ trialNumbers + C(scarce)"
    
    # Fitting a single-slope model and a two-slope model to see which is better
    if mixed:
        glm_1 = smf.mixedlm(formula=two_slope_formula, data=relevant_trials, groups=relevant_trials['workerId'])
        glm_2 = smf.mixedlm(formula=one_slope_formula, data=relevant_trials, groups=relevant_trials['workerId'])
    else:
        glm_1 = smf.glm(formula=two_slope_formula, data=relevant_trials)
        glm_2 = smf.glm(formula=one_slope_formula, data=relevant_trials)

    try:
        results_1 = glm_1.fit()
        results_2 = glm_2.fit()
    except:
        continue
        
    bic1 = results_1.bic_llf
    bic2 = results_2.bic_llf
    
    # print(results_1.summary())
    
    # Saving the slopes and the single intercept of both phases from the two-slope model
    b1 = results_1.params['trialNumbers']
    b2 = results_1.params['trialNumbers'] + results_1.params['trialNumbers:C(isPerformanceTrial)[T.1]']
    i1 = results_1.params['Intercept']
    i2 = results_1.params['Intercept'] + results_1.params['C(isPerformanceTrial)[T.1]']
    
    b3 = results_2.params['trialNumbers']
    i3 = results_2.params['Intercept']
    if(bic1 < bic2):
        # Save data about significant partitions, where the bic of the two-slope model is higher than one-slope
        sig_bic_scores_control.append((trialNum, bic1))
        
    x1 = all_trial_nos_control[0:len(partition_1_mean_scores)]
    x2 = all_trial_nos_control[len(partition_1_mean_scores):len(partition_1_mean_scores)+len(partition_2_mean_scores)]
    control_plot_data.append((
        trialNum,
        x1,
        x2,
        partition_1_mean_scores,
        partition_2_mean_scores,
        b1,
        b2,
        i1,
        i2,
        b3, 
        i3,
        partition_1_df,
        partition_2_df
    ))
    control_glm_results.append((trialNum, results_1, results_2))

In [ ]:
sig_bic_scores_control

In [ ]:
# Finding the best partition of all significant ones 

if use_model_data:
    partition_candidates = [9] # Use partition from human data
else:
    sig_bic_scores_control = np.array(sig_bic_scores_control)

    g_range = np.arange(0, 1, 0.0001)
    best_partitions = []

    for gamma in g_range:
        # Minimum bic
        partition_scores = sig_bic_scores_control[:,1]
        best_partition = sig_bic_scores_control[np.argmin(partition_scores),0]
        best_partitions.append(best_partition)

    partition_candidates = np.unique(best_partitions)
    print("Partition candidates: ")
    print(partition_candidates)
    plt.plot(g_range, best_partitions)
    plt.xlabel("Gamma (Decay Factor)")
    plt.ylabel("Best Partition")
    plt.show()


In [ ]:
# Showing GLM results - control
candidate_glm_results_control = [data for data in control_glm_results if float(data[0]) in partition_candidates]

for trialNum, results_1, results_2 in candidate_glm_results_control:
    print("Trial Num: {}".format(trialNum))
    print("\nTwo-Slope Model:\nBIC LLF: {0:0.4f}\n\nBIC Deviance: {1:0.4f}\n\nAIC: {2:0.4f}\n"
          .format(results_1.bic_llf,results_1.bic_deviance, results_1.aic))
    print(results_1.summary())
    
    print("\nOne-Slope Model:\nBIC LLF: {0:0.4f}\n\nBIC Deviance: {1:0.4f}\n\nAIC: {2:0.4f}\n"
          .format(results_2.bic_llf,results_2.bic_deviance, results_2.aic))
    print(results_2.summary())


In [ ]:
# Plotting candidate partitions - control
candidate_plot_data_control = [data for data in control_plot_data if float(data[0]) in partition_candidates]
#significant_plot_data_control = [data for data in control_plot_data if float(data[0]) in sig_bic_scores_control[:,0]]

which_plot = "candidates" #all, significant, candidates

if which_plot == "candidates":
    plot_data = candidate_plot_data_control
elif which_plot == "significant":
    plot_data = significant_plot_data_control
else:
    plot_data = control_plot_data
    
plot_single_slope = False
plot_two_slope = True
scatter = False

for trialNum, x1, x2, y1, y2, b1, b2, i1, i2,b3, i3, df_1, df_2 in plot_data:
    plt.figure()
    # Expected Score curves
    plt.plot(all_trial_nos_control, all_trials_mean_control, alpha=0.2, color='k')
    plt.plot(x1, y1, alpha=0.5)
    plt.plot(x2, y2, alpha=0.5)

    # Partition line
    plt.axvline(trialNum, linestyle="--", color='k',alpha=0.5)
    
    # Scatter points
    if scatter:
        plt.scatter(df_1['trialNumbers'], df_1['expectedScores'], color='b', marker='o', alpha=0.1,s=0.8)
        plt.scatter(df_2['trialNumbers'], df_2['expectedScores'], color='orange', marker='o', alpha=0.1,s=0.8)

    # Slope lines of one-slope model
    if plot_single_slope:
        x3 = np.concatenate([x1,x2],axis=0)
        plt.plot(x3, np.array(x3) * b3 + i3, color='g', alpha=0.5)
    
    if plot_two_slope:
        # Slope lines of two-slope model
        plt.plot(x1, np.array(x1) * b1 + i1, color='b')
        plt.plot(x2, np.array(x2) * b2 + i2, color='orange')
    
    plt.title("Trial: {}".format(trialNum))
    
plt.show()

In [ ]:
control_partition = partition_candidates[0] # Somehow choose based on possible candidates

# Separating into learning and performance phase
control_learning_data = analysis_data.loc[(analysis_data.scarce == 0) & (analysis_data.trialNumbers <= control_partition)]
control_performance_data = analysis_data.loc[(analysis_data.scarce == 0) & (analysis_data.trialNumbers > control_partition)]

# Without transformation
control_learning_data_orig = filtered_data.loc[(filtered_data.scarce == 0) & (filtered_data.trialNumbers <= control_partition)]
control_performance_data_orig = filtered_data.loc[(filtered_data.scarce == 0) & (filtered_data.trialNumbers > control_partition)]

# Store the slope, intercept, etc. of the best partition
control_partition_plot_data = [data for data in candidate_plot_data_control if data[0] == control_partition][0]

# 1.2 - Linear Regression: Response Var vs. Trial Number x Condition

Results for response variable "expectedScores" reported in paper

In [ ]:
# Select the response variable for this analysis

response_var = "expectedScores"
assert (response_var in response_vars), "{} is not a valid response variable".format(response_var)

In [ ]:
# Flag to compare entire performance data or only upto 30 trials, where both conditions are comparable

compare_all = True
num_max_trials = min(
    analysis_data[analysis_data.scarce==1].trialNumbers.max(),
    analysis_data[analysis_data.scarce==0].trialNumbers.max()
)

In [ ]:
# Learning Phase

use_orig = True

if use_orig:
    learning_df = pd.concat([scarce_learning_data_orig, control_learning_data_orig]).reset_index(drop=True)
else: 
    learning_df = pd.concat([scarce_learning_data, control_learning_data]).reset_index(drop=True)

# Compare only the first 30 trials
if not compare_all:
    learning_df = learning_df.loc[learning_df.trialNumbers <= num_max_trials]

formula = '{} ~ trialNumbers + C(scarce) + trialNumbers:C(scarce)'.format(response_var)

mixed = True

if mixed:
    glm = smf.mixedlm(formula=formula, data=learning_df, groups=learning_df['workerId'])
else:
    glm = smf.glm(formula=formula, data=learning_df)

results = glm.fit()

print(results.summary())
resids = results.resid_response if not mixed else results.resid
normaltest = scipy.stats.normaltest(resids)
plt.hist(resids, density=True)
plt.title("Normal Test: {0:0.4f}, p: {1:0.4f}".format(normaltest[0], normaltest[1]))
plt.show()

In [ ]:
# Plotting the data of both learning phases

x1 = control_partition_plot_data[1]
x2 = scarce_partition_plot_data[1]
y1 = learning_df[learning_df.scarce==0].groupby("trialNumbers").mean()[response_var]
y2 = learning_df[learning_df.scarce==1].groupby("trialNumbers").mean()[response_var]
b1 = results.params['trialNumbers']
b2 = results.params['trialNumbers'] + results.params['trialNumbers:C(scarce)[T.1]']
i1 = results.params['Intercept']
i2 = results.params['Intercept'] + results.params['C(scarce)[T.1]']

plt.plot(x2, y2, color='b', alpha=0.5, label="Scarce")
plt.plot(x1, y1, color='orange', alpha=0.5, label="Control")


# Slope lines
plt.plot(x1, np.array(x1) * b1 + i1, color='orange')
plt.plot(x2, np.array(x2) * b2 + i2, color='b')
plt.title("Learning Phases - Learning Curves - {}".format(response_var))
plt.xlabel("Trial Number")
plt.ylabel("{} - {}".format(response_var, "(Not Transformed)" if use_orig else "(Transformed)"))
plt.legend()


plt.show()



In [ ]:
# Performance Phase

use_orig = True

if use_orig:
    performance_df = pd.concat([scarce_performance_data_orig, control_performance_data_orig]).reset_index(drop=True)
else: 
    performance_df = pd.concat([scarce_performance_data, control_performance_data]).reset_index(drop=True)

if not compare_all:
    performance_df = performance_df.loc[performance_df.trialNumbers <= num_max_trials]
    
formula = '{} ~ trialNumbers + C(scarce) + trialNumbers:C(scarce)'.format(response_var)

mixed = True

if mixed:
    glm = smf.mixedlm(formula=formula, data=performance_df, groups=performance_df['workerId'])
else:
    glm = smf.glm(formula=formula, data=performance_df)

results = glm.fit()

print(results.summary())
resids = results.resid_response if not mixed else results.resid
normaltest = scipy.stats.normaltest(resids)
plt.hist(resids, density=True)
plt.title("Normal Test: {0:0.4f}, p: {1:0.4f}".format(normaltest[0], normaltest[1]))
plt.show()

In [ ]:
# Plotting the data of both performance phases together

x1 = control_partition_plot_data[2]
x2 = scarce_partition_plot_data[2]
y1 = performance_df[performance_df.scarce==0].groupby("trialNumbers").mean()[response_var]
y2 = performance_df[performance_df.scarce==1].groupby("trialNumbers").mean()[response_var]
b1 = results.params['trialNumbers']
b2 = results.params['trialNumbers'] + results.params['trialNumbers:C(scarce)[T.1]']
i1 = results.params['Intercept']
i2 = results.params['Intercept'] + results.params['C(scarce)[T.1]']

if not compare_all:
    end_index = np.where(x1 == num_max_trials)[0]
    if len(end_index) > 0:
        end_index = end_index[0]
        x1 = x1[0:end_index]
        y1 = y1[0:end_index]
    
    end_index = np.where(x2 == num_max_trials)[0]
    if len(end_index) > 0:
        end_index = end_index[0]
        x2 = x2[0:end_index]
        y2 = y2[0:end_index]
        

plt.plot(x2, y2, color='b', alpha=0.5, label="Scarce")
plt.plot(x1, y1, color='orange', alpha=0.5, label="Control")


# Slope lines
plt.plot(x1, np.array(x1) * b1 + i1, color='orange')
plt.plot(x2, np.array(x2) * b2 + i2, color='b')
plt.title("Performance Phases - Learning Curves - {}".format(response_var))
plt.xlabel("Trial Number")
plt.ylabel("{} - {}".format(response_var, "(Not Transformed)" if use_orig else "(Transformed)"))
plt.legend()

plt.show()

# 1.3 - Linear Regression: Response Var vs. Num Rewarded Trials

GLM Analysis for the effect of number of previously rewarded trials and previously unrewarded trials on improvement in response variable in each condition

(response_var) ~ numRewardedTrials + numUnrewardedTrials + condition + numRewardedTrials x condition

Results for response variable "expectedScores" reported in paper

In [ ]:
# Select the response variable for this analysis

response_var = "expectedScores"
assert (response_var in response_vars), "{} is not a valid response variable".format(response_var)

In [ ]:
# Flag to compare entire performance data or only upto 30 trials, where both conditions are comparable

compare_all = True
num_max_trials = min(
    analysis_data[analysis_data.scarce==1].trialNumbers.max(),
    analysis_data[analysis_data.scarce==0].trialNumbers.max()
)

In [ ]:
# Learning Phase 

use_orig = True

if use_orig:
    learning_df = pd.concat([scarce_learning_data_orig, control_learning_data_orig]).reset_index(drop=True)
else: 
    learning_df = pd.concat([scarce_learning_data, control_learning_data]).reset_index(drop=True)

if not compare_all:
    learning_df = learning_df.loc[learning_df.trialNumbers <= num_max_trials]
    
formula = '{} ~ numRewardedTrials + C(scarce) + numUnrewardedTrials + numRewardedTrials:C(scarce)'.format(response_var)

mixed = True

if mixed:
    glm = smf.mixedlm(formula=formula, data=learning_df, groups=learning_df['workerId'])
else:
    glm = smf.glm(formula=formula, data=learning_df)

results_learning = glm.fit()

print(results_learning.summary())
resids = results_learning.resid_response if not mixed else results_learning.resid
normaltest = scipy.stats.normaltest(resids)
plt.hist(resids, density=True)
plt.title("Normal Test: {0:0.4f}, p: {1:0.4f}".format(normaltest[0], normaltest[1]))
plt.show()

In [ ]:
# comparing coefficients using contrasts

# between conditions
# H_a = numRewardedTrials - numUnrewardedTrials > 0
print("H_a = numRewardedTrials - numUnrewardedTrials > 0")
r = np.zeros_like(results_learning.params)
r[list(results_learning.params.index).index("numRewardedTrials")] = 1
r[list(results_learning.params.index).index("numUnrewardedTrials")] = -1

print(results_learning.t_test(np.expand_dims(r[:-1], axis=0)))
print(results_learning.f_test(r))
print(results_learning.wald_test(r))

# within scarce condition
# H_a = numRewardedTrials + numRewardedTrial:C(scarce)[T.1] - numUnrewardedTrials > 0
print("\nH_a = numRewardedTrials + numRewardedTrial:C(scarce)[T.1] - numUnrewardedTrials > 0")
r = np.zeros_like(results_learning.params)
r[list(results_learning.params.index).index("numRewardedTrials")] = 1
r[list(results_learning.params.index).index("numRewardedTrials:C(scarce)[T.1]")] = 1
r[list(results_learning.params.index).index("numUnrewardedTrials")] = -1
print(results_learning.t_test(np.expand_dims(r[:-1], axis=0)))
print(results_learning.f_test(r))
print(results_learning.wald_test(r))

In [ ]:
# Performance phase

use_orig = True

if use_orig:
    performance_df = pd.concat([scarce_performance_data_orig, control_performance_data_orig]).reset_index(drop=True)
else: 
    performance_df = pd.concat([scarce_performance_data, control_performance_data]).reset_index(drop=True)

if not compare_all:
    performance_df = performance_df.loc[performance_df.trialNumbers <= num_max_trials]
    
formula = '{} ~ numRewardedTrials + C(scarce) + numUnrewardedTrials + numRewardedTrials:C(scarce)'.format(response_var)

mixed = True

if mixed:
    glm = smf.mixedlm(formula=formula, data=performance_df, groups=performance_df['workerId'])
else:
    glm = smf.glm(formula=formula, data=performance_df)

results_performance = glm.fit()

print(results_performance.summary())
resids = results_performance.resid_response if not mixed else results_performance.resid
normaltest = scipy.stats.normaltest(resids)
plt.hist(resids, density=True)
plt.title("Normal Test: {0:0.4f}, p: {1:0.4f}".format(normaltest[0], normaltest[1]))
plt.show()

In [ ]:
# comparing coefficients using contrasts

# between conditions
# H_a = numRewardedTrials - numUnrewardedTrials > 0
print("H_a = numRewardedTrials - numUnrewardedTrials > 0")
r = np.zeros_like(results_performance.params)
r[list(results_performance.params.index).index("numRewardedTrials")] = 1
r[list(results_performance.params.index).index("numUnrewardedTrials")] = -1

print(results_performance.t_test(np.expand_dims(r[:-1], axis=0)))
print(results_performance.f_test(r))
print(results_performance.wald_test(r))

# within scarce condition
# H_a = numRewardedTrials + numRewardedTrial:C(scarce)[T.1] - numUnrewardedTrials > 0
print("\nH_a = numRewardedTrials + numRewardedTrial:C(scarce)[T.1] - numUnrewardedTrials > 0")
r = np.zeros_like(results_performance.params)
r[list(results_performance.params.index).index("numRewardedTrials")] = 1
r[list(results_performance.params.index).index("numRewardedTrials:C(scarce)[T.1]")] = 1
r[list(results_performance.params.index).index("numUnrewardedTrials")] = -1
print(results_performance.t_test(np.expand_dims(r[:-1], axis=0)))
print(results_performance.f_test(r))
print(results_performance.wald_test(r))

# 1.4 - Independent Samples T-test

Results for response variable "expectedScores" reported in paper

In [ ]:
# Select the response variable for this analysis

response_var = "expectedScores"
assert (response_var in response_vars), "{} is not a valid response variable".format(response_var)

In [ ]:
# Flag to compare entire performance data or only upto 30 trials, where both conditions are comparable

compare_all = False
num_max_trials = min(
    analysis_data[analysis_data.scarce==1].trialNumbers.max(),
    analysis_data[analysis_data.scarce==0].trialNumbers.max()
)

In [ ]:
use_transformed = False

if not compare_all:
    control_performance_data = control_performance_data.loc[control_performance_data.trialNumbers <= num_max_trials]
    scarce_performance_data = scarce_performance_data.loc[scarce_performance_data.trialNumbers <= num_max_trials]

if use_transformed:
    # Using the transformed data divided into phases
    control_averages = control_performance_data.groupby('workerId').mean()
    scarce_averages = scarce_performance_data.groupby('workerId').mean()
else:
    # Using the original data divided into phases
    control_averages = control_performance_data_orig.groupby('workerId').mean()
    scarce_averages = scarce_performance_data_orig.groupby('workerId').mean()
    

In [ ]:
plt.figure()
plt.plot(np.sort(np.unique(control_performance_data.trialNumbers)), control_performance_data.groupby('trialNumbers').mean()[response_var], color='orange')
plt.plot(np.sort(np.unique(scarce_performance_data.trialNumbers)), scarce_performance_data.groupby('trialNumbers').mean()[response_var], color='b')
plt.show()

In [ ]:

# Plotting normality for both groups

fig, ax = plt.subplots(nrows=1,ncols=2,figsize=(10, 5),squeeze=False)
plt.suptitle("Distributions of Averages - {}".format(response_var))
ax = ax.flatten()

for idx, group in enumerate([(control_averages, "Control"), (scarce_averages, "Scarce")]):
    data = group[0][response_var]
    label = group[1]
    print(data.mean(), data.std())
    normaltest = scipy.stats.normaltest(data)
    density = scipy.stats.gaussian_kde(data)
    n, x, _ = ax[idx].hist(data, density=True)
    ax[idx].set_title(label)
    ax[idx].plot(x, density(x))
    ax[idx].text(0.75,0.9, "S: {0:0.4f}\nP: {1:0.4f}".format(normaltest.statistic, normaltest.pvalue), transform=ax[idx].transAxes)

plt.subplots_adjust(
                wspace=0.4, 
                hspace=0.4)
plt.show()

In [ ]:
# Perform independent T-test if data is normal


two = scipy.stats.ttest_ind(scarce_averages[response_var], control_averages[response_var], equal_var=True, alternative='two-sided')
less = scipy.stats.ttest_ind(scarce_averages[response_var], control_averages[response_var], equal_var=True, alternative='less')
greater = scipy.stats.ttest_ind(scarce_averages[response_var], control_averages[response_var], equal_var=True, alternative='greater')

print("Response Variable: {}".format(response_var))
print("Two-Sided:")
print("\tStatistic: {0:0.3f}\n\tP-value: {1}".format(two.statistic, two.pvalue))
print("Scarce < Control:")
print("\tStatistic: {0:0.3f}\n\tP-value: {1}".format(less.statistic, less.pvalue))
print("Scarce > Control:")
print("\tStatistic: {0:0.3f}\n\tP-value: {1}".format(greater.statistic, greater.pvalue))

In [ ]:
# Perform Mann-Whitney U test if data is not normal

two = scipy.stats.mannwhitneyu(scarce_averages[response_var], control_averages[response_var], alternative='two-sided')
less = scipy.stats.mannwhitneyu(scarce_averages[response_var], control_averages[response_var], alternative='less')
greater = scipy.stats.mannwhitneyu(scarce_averages[response_var], control_averages[response_var], alternative='greater')

print("Two-Sided:")
print("\tStatistic: {0:0.3f}\n\tP-value: {1}".format(two.statistic, two.pvalue))
print("Scarce < Control:")
print("\tStatistic: {0:0.3f}\n\tP-value: {1}".format(less.statistic, less.pvalue))
print("Scarce > Control:")
print("\tStatistic: {0:0.3f}\n\tP-value: {1}".format(greater.statistic, greater.pvalue))

# 1.6 - Stable Strategy Comparisons

Analysis to see which of the strategies participants converged on and whether they are the same in both conditions

Not reported in paper

In [ ]:
filtered_data['strategy'] = filtered_data['strategy'].astype('int64')

In [ ]:
# Strategies taken are from the latter 50% of the trials of each participant

# Take median of participant strategies or mode (most frequent)
measure = 'median' #median, mode

if measure == 'median':
    scarce_end_strat = filtered_data.loc[(filtered_data.scarce == 1) & (filtered_data.trialNumbers > 60)].groupby('workerId').median()['strategy'].astype('int64')
    control_end_strat = filtered_data.loc[(filtered_data.scarce == 0) & (filtered_data.trialNumbers > 15)].groupby('workerId').median()['strategy'].astype('int64')
elif measure == 'mode':
    scarce_end_strat = filtered_data.loc[(filtered_data.scarce == 1) & (filtered_data.trialNumbers > 60)].groupby('workerId')['strategy'].agg(pd.Series.mode)#.astype('int64')
    control_end_strat = filtered_data.loc[(filtered_data.scarce == 0) & (filtered_data.trialNumbers > 15)].groupby('workerId')['strategy'].agg(pd.Series.mode)#.astype('int64')

In [ ]:
# Create contingency table

table = np.zeros((89,2))

for strat in scarce_end_strat:
    #print(type(strat))
    if type(strat) != int and type(strat) != np.int64:
        strat = strat[0]
    table[strat][1] += 1
    
for strat in control_end_strat:
    if type(strat) != int and type(strat) != np.int64:
        strat = strat[0]
    table[strat][0] += 1
    
table

In [ ]:
# Perform chi square test of independence

summed_tab = table.sum(axis=1)
nz_rows = np.where(summed_tab != 0)[0]
new_table = table[nz_rows]



crosstab = pd.crosstab(new_table[:,0], new_table[:,1])
scipy.stats.chi2_contingency(new_table)
